<a href="https://colab.research.google.com/github/financieras/python_poo/blob/main/tablero5/manadas_aleatorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno de tablero5/main2.py con modificaciones

In [14]:
import random
import os
import time

# Constants
RED = "\033[1;91m"        # Red Bold High Intensty
GREEN = "\033[1;92m"      # Green Bold High Intensty
YELLOW = "\033[1;93m"     # Yellow Bold High Intensty
BLUE = "\033[1;94m"       # Blue Bold High Intensty
PURPLE = "\033[1;95m"     # Purple Bold High Intensty
CYAN = "\033[1;96m"       # Cyan Bold High Intensty
RESET = "\033[0m"



class Jugador:
    def __init__(self, letra, x, y):
        self.letra = letra
        self.x = x
        self.y = y
        self.puntaje = 0

    def __str__(self):
        return f'{self.letra}'    #f'{self.letra}: ({self.x}, {self.y})'

    def puntos_jugador(self):
        return f'{self.letra}: {self.puntaje}'

class Comida:
    def __init__(self, x, y, valor):
        self.x = x
        self.y = y
        self.valor = valor


class Tablero:
    def __init__(self, DIM, num_jugadores, num_comida):
        self.DIM = DIM
        self.tablero = [['·']*DIM for _ in range(DIM)]
        self.jugadores = []     # array de objetos de la clase Jugador
        self.comidas = []       # array de objetos de la clase Comida
        # Creando los jugadores iniciales sobre el tablero
        for i in range(num_jugadores):  # para cada jugador desde i=0,1,2, ...
            while True: # buscamos coordenadas para el nuevo jugador que no caigan sobre otro
                jugador_x = random.randint(0, DIM-1)    # componente x del jugador
                jugador_y = random.randint(0, DIM-1)    # componente y del jugador
                if self.tablero[jugador_x][jugador_y] == '·':   # si las coordenadas caen en un punto es que no hay otro jugador en esas coordenadas
                    break
            self.tablero[jugador_x][jugador_y] = chr(65 + i)    # se asigna una letra mayúscula correlativa: A, B, C, ...
            self.jugadores.append(Jugador(chr(65 + i), jugador_x, jugador_y))   # se crea el objeto Jugador y se añade al array de jugadores
        for i in range(num_comida):   # para cada celda de comida desde i=0,1,2, ...
            while True:
                comida_x = random.randint(0, DIM-1)     # componente x de la celda i-ésima con comina
                comida_y = random.randint(0, DIM-1)     # componente y de la celda i-ésima con comida
                if self.tablero[comida_x][comida_y] == '·':     # si las coordenadas caen en un punto es que no caen en un jugador o en otra comida
                    break
            valor_comida = random.randint(1, 9)         # la cantidad de comida que contiene una celda de comida varía aleatoriamente entre 1 y 9, viene representada por ese número
            self.tablero[comida_x][comida_y] = str(valor_comida)    # se modifica el valor de la celda, pasa de ser un punto a ser un número en forma de str
            self.comidas.append(Comida(comida_x, comida_y, valor_comida))   # Se crea un objeto de la clase Comida y se añade a la lista comidas

    def __str__(self):      # representamos el objeto tablero
        res = ''            # inicializamos como vacío el string que retornará la respuesta que imprime el tablero
        for fila in self.tablero:                   # por cada fila de la matriz tablero
            for cont, caracter in enumerate(fila):  # por cada caracter de la fila
                if caracter in [chr(i) for i in range(65, 91)]:     # si el caracter de la matriz tablero es una letra mayúscula
                    fila[cont] = f'{RED}{caracter}{RESET}'   # cambia el caracter letra mayúscula por ella misma con color, solo para imprimir
            res += ' '.join(fila) + '\n'    # concatena todos los elementos de una fila y añade un retorno de carro al final de la fila
        return res

    def mover_jugador(self, jugador):
        movimientos = [(0, 1), (0, -1), (1, 0), (-1, 0)]    # tuplas: arriba, abajo, derecha, izquierda
        libertades = [True] * 4   # Inicialmente suponemos que el jugador tiene todas las libertades de movimiento
        for count, movimiento in enumerate(movimientos):
            dx, dy = movimiento
            nueva_x = jugador.x + dx
            nueva_y = jugador.y + dy
            if not (0 <= nueva_x < self.DIM and 0 <= nueva_y < self.DIM) or self.tablero[nueva_x][nueva_y].isalpha():
                libertades[count] = False   # jugador ahogado en esa dirección, no puede moverse al tener pared u otro jugador
        if any(libertades):  # da False cuando el jugador está ahogado en todas las direcciones, y entonces pasa turno
            dx, dy = random.choice(movimientos)
            nueva_x = jugador.x + dx
            nueva_y = jugador.y + dy
            while not (0 <= nueva_x < self.DIM and 0 <= nueva_y < self.DIM) or self.tablero[nueva_x][nueva_y].isalpha():
                dx, dy = random.choice(movimientos)
                nueva_x = jugador.x + dx
                nueva_y = jugador.y + dy
            if self.tablero[nueva_x][nueva_y].isdigit():
                valor_comida = int(self.tablero[nueva_x][nueva_y])
                jugador.puntaje += valor_comida
                self.comidas = [comida for comida in self.comidas if not (comida.x == nueva_x and comida.y == nueva_y)]

            self.tablero[jugador.x][jugador.y] = '·'
            jugador.x = nueva_x
            jugador.y = nueva_y
            self.tablero[jugador.x][jugador.y] = jugador.letra


class Juego:
    def __init__(self, DIM, num_jugadores, num_comida):
        self.tablero = Tablero(DIM, num_jugadores, num_comida)      # creamos el objeto tablero dando su dimensión, número de jugadores y número de comida

    def imprimir_ranking(self):
        jugadores_ordenados = sorted(self.tablero.jugadores, key=lambda jugador: jugador.puntaje, reverse=True)
        print("Ranking de jugadores:")
        for jugador in jugadores_ordenados:
            print(jugador.puntos_jugador()) # Equivale a poner: print(f"{jugador.letra}: {jugador.puntaje}")

    def jugar(self):
        print(self.tablero)             # se imprime el primer tablero
        while self.tablero.comidas:     # mientras haya comida en el array de comidas
            for jugador in self.tablero.jugadores:
                if not self.tablero.comidas:
                    break   # si ya no hay comida en el tablero finaliza el juego sin terminar el turno de los otros jugadores
                self.tablero.mover_jugador(jugador)
                #os.system('cls' if os.name == 'nt' else 'clear')
                print(self.tablero)     # se imprimen los sucesivos tableros
                print()
                #time.sleep(0.1)
        ganador = max(self.tablero.jugadores, key=lambda j: j.puntaje)
        print(f"¡El ganador es el jugador {ganador.letra} con {ganador.puntaje} puntos!\n")
        self.imprimir_ranking()

if "__main__" == __name__:
    DIM = 5
    num_jugadores = 7
    num_comida = 10  # cantidad de celdas que contienen comida
    partida = Juego(DIM, num_jugadores, num_comida)
    partida.jugar()

9 7 · · A
· · 5 · 2
D 6 7 · F
G 7 5 C E
· · 4 B 5

9 7 · · ·
· · 5 · A
D 6 7 · F
G 7 5 C E
· · 4 B 5


9 7 · · ·
· · 5 · A
D 6 7 · F
G 7 5 C E
· · B · 5


9 7 · · ·
· · 5 · A
D 6 7 · F
G 7 C · E
· · B · 5


9 7 · · ·
D · 5 · A
· 6 7 · F
G 7 C · E
· · B · 5


9 7 · · ·
D · 5 · A
· 6 7 · E
G 7 C · ·
· · B · 5


9 7 · · ·
D · 5 · A
· 6 7 · ·
G 7 C · F
· · B · 5


9 7 · · ·
D · 5 · A
· 6 7 · ·
· 7 C · F
G · B · 5


9 7 · · A
D · 5 · ·
· 6 7 · ·
· 7 C · F
G · B · 5


9 7 · · A
D · 5 · ·
· 6 7 · ·
· 7 C · F
G · · B 5


9 7 · · A
D · 5 · ·
· 6 7 · ·
· C · · F
G · · B 5


D 7 · · A
· · 5 · ·
· 6 7 · ·
· C · · F
G · · B 5


D 7 · · A
· · 5 · E
· 6 7 · ·
· C · · F
G · · B 5


D 7 · · A
· · 5 · E
· 6 7 · F
· C · · ·
G · · B 5


D 7 · · A
· · 5 · E
· 6 7 · F
G C · · ·
· · · B 5


D 7 · · ·
· · 5 · A
· 6 7 · F
G C · · ·
· · · B 5


D 7 · · ·
· · 5 · A
· 6 7 · F
G C · · ·
· · B · 5


D 7 · · ·
· · 5 · A
· 6 7 · F
G · · · ·
· C B · 5


· 7 · · ·
D · 5 · A
· 6 7 · F
G · · · ·
· C B · 5


· 7 · · ·
D ·